In [112]:
!pip install -q pyomo

In [113]:
from pyomo.environ import * 

In [114]:
from timeit import default_timer as timer

In [115]:
import numpy as np
import pandas as pd


In [116]:
data = pd.read_csv("Knapsack.txt",delimiter = " ",header=None)
data.columns = ["values","weights"]

In [117]:
data

values  weights
0       26      550
1       91       84
2       72       83
3       90       43
4       46        4
5       55       44
6        8        6
7       35       82
8       75       92
9       61       25
10      15       83
11      77       56
12      40       18
13      63       58
14      75       14
15      29       48
16      75       70
17      17       96
18      78       32
19      40       68
20      44       92
21      40       60
22      20       55
23      23       12
24      55       15
25      65       38
26      12       40

In [118]:
start_time = timer()

In [119]:
model = ConcreteModel()

In [120]:
model.constrains = ConstraintList()

In [121]:
objec_coef = np.array(data.iloc[:,0])

In [122]:
constr_coef = np.array(data.iloc[:,1])

In [123]:
M = data.iloc[0,0]
N = data.iloc[0,1]

In [124]:
model.x = Var(np.arange(M),domain = Binary)


In [125]:
model.constrains.add(sum(constr_coef[j]*model.x[j] for j in range(M)) <= N)


In [126]:
model.objective = Objective(expr = sum(objec_coef[k]*model.x[k] for k in range(M)),sense = maximize)

In [127]:
model.pprint()

2 Set Declarations
    constrains_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}

1 Var Declarations
    x : Size=26, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Bina

In [128]:
!apt-get install -y -qq coinor-cbc

In [129]:
opt_cbc = SolverFactory('cbc')

In [130]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [131]:
end_time = timer()

In [132]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in range(M):
  print('x[',i+1,'] = ', model.x[i].value)

print('\nConstraints')
model.constrains.display()


Objective =  911.0

Decision Variables
x[ 1 ] =  0.0
x[ 2 ] =  1.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  0.0
x[ 9 ] =  0.0
x[ 10 ] =  1.0
x[ 11 ] =  0.0
x[ 12 ] =  1.0
x[ 13 ] =  1.0
x[ 14 ] =  0.0
x[ 15 ] =  1.0
x[ 16 ] =  0.0
x[ 17 ] =  1.0
x[ 18 ] =  0.0
x[ 19 ] =  1.0
x[ 20 ] =  0.0
x[ 21 ] =  0.0
x[ 22 ] =  0.0
x[ 23 ] =  0.0
x[ 24 ] =  1.0
x[ 25 ] =  1.0
x[ 26 ] =  1.0

Constraints
constrains : Size=1
    Key : Lower : Body  : Upper
      1 :  None : 544.0 : 550.0


3. [R] Find the optimal subset of items to include in knapsack for this problem.

In [133]:
for i in range(M):
  if model.x[i].value == 1:
    print(model.x[i])
  
  

x[1]
x[2]
x[3]
x[4]
x[5]
x[6]
x[9]
x[11]
x[12]
x[14]
x[16]
x[18]
x[23]
x[24]
x[25]


Report the solution and note the solution time.

In [134]:
print('Time taken to solve problem:',end_time - start_time)

Time taken to solve problem: 2.505030705999161


5. [R] If it seems that the time to solve the problem is more, try to solve the problem in lesser time complexity
and report the solution and time. (Hint- dynamic programming)

In [135]:
import time
start = time.time()
x=[]
for i in range(M):
  x.append((constr_coef[i],objec_coef[i]))
N = 550


new_table = [[0 for w in range(N+1)] for i in range(len(x)+1)]

# Fill the new_table using dynamic programming
for i in range(1, len(x)+1):
    for w in range(1, N+1):
        if x[i-1][0] > w:
            
            new_table[i][w] = new_table[i-1][w]
        else:
            # Find the maximum value with or without item i
            value_with = x[i-1][1] + new_table[i-1][w-x[i-1][0]]
            value_without = new_table[i-1][w]
            new_table[i][w] = max(value_with, value_without)

# Find the optimal subset of x to include in the knapsack
included_x = []
i, w = len(x), N
while i > 0 and w > 0:
    if new_table[i][w] != new_table[i-1][w]:
        included_x.append(i-1)
        w -= x[i-1][0]
    i -= 1
included_x.reverse()


print("Optimal optimal values of x")
for item in included_x:
    print("x[",item,"]")
print("Total value of the knapsack:", new_table[-1][-1])
end = time.time()


Optimal optimal values of x
x[ 1 ]
x[ 2 ]
x[ 3 ]
x[ 4 ]
x[ 5 ]
x[ 6 ]
x[ 9 ]
x[ 11 ]
x[ 12 ]
x[ 14 ]
x[ 16 ]
x[ 18 ]
x[ 23 ]
x[ 24 ]
x[ 25 ]
Total value of the knapsack: 911


In [136]:
print("total time taken",end - start)

total time taken 0.025999069213867188
